In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
import sys, os
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
from _global.bbx import *
sys.path.extend([RESEARCH])
from object_detection.utils import config_util
from object_detection.builders import model_builder
from IPython import get_ipython
import cv2 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
ipython = get_ipython()

In [ ]:
def find_and_extract_face_from_img(img, model, output_path_with_filename=None):
    shape = img.shape
    prediction = predict_for_image(np.array(img), model)

    best_bbx = get_best_bbx(prediction['detection_boxes'],prediction['detection_scores'])
    best_bbx[0]*= shape[0]
    best_bbx[1] *= shape[1]
    best_bbx[2] *= shape[0]
    best_bbx[3] *= shape[1]

    face = img[int(shape[0] -best_bbx[2]):int(shape[0] -best_bbx[0]), int(best_bbx[1]):int(best_bbx[3])]
    face = cv2.resize(face, (RECOG_IMAGE_SIZES[1], RECOG_IMAGE_SIZES[0]))
    if output_path_with_filename == None:
        return face
    cv2.imwrite(output_path_with_filename,face)

In [ ]:
def find_and_extract_faces(input_path, output_path, model):
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    imgs = [(f ,cv2.imread(os.path.join(input_path, f))) for f in os.listdir(input_path)]
    for i in range(len(imgs)):
        filename, img = imgs[i]
        find_and_extract_face(img, model, os.path.join(output_path, filename))